In [40]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

Using cuda


In [42]:
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono'], low_memory=False)
df_original.dropna(inplace=True)
df_original.index = pd.to_datetime(df_original['Data e Hora'], format='%Y-%m-%d %H:%M:%S')

df_original['Data e Hora'] = pd.to_datetime(df_original['Data e Hora'])

df_original['year'] = df_original['Data e Hora'].dt.year
df_original['month'] = df_original['Data e Hora'].dt.month
df_original['day'] = df_original['Data e Hora'].dt.day
df_original['hour'] = df_original['Data e Hora'].dt.hour
df_original['minute'] = df_original['Data e Hora'].dt.minute
df_original['dayofyear'] = df_original['Data e Hora'].dt.dayofyear

df_original['hour_sin'] = np.sin(2 * np.pi * df_original['hour'] / 24)
df_original['hour_cos'] = np.cos(2 * np.pi * df_original['hour'] / 24)
df_original['dayofyear_sin'] = np.sin(2 * np.pi * df_original['dayofyear'] / 365)
df_original['dayofyear_cos'] = np.cos(2 * np.pi * df_original['dayofyear'] / 365)

# df = df_raw[['PM2.5', 'hour_sin', 'hour_cos', 'dayofyear_sin', 'dayofyear_cos', 'PM10', 'Monóxido de Carbono' ]]
df = df_original[['PM2.5', 'PM10', 'Monóxido de Carbono']]
df

,PM2.5,PM10,Monóxido de Carbono
Data e Hora,,,
2015-11-04 10:30:00,25.0,39.41,0.43
2015-11-04 11:30:00,13.0,38.38,0.42
2015-11-04 12:30:00,12.0,24.91,0.57
2015-11-04 13:30:00,14.0,32.16,0.55
2015-11-04 14:30:00,12.0,26.99,0.56
...,...,...,...
2022-12-31 17:30:00,3.0,5.25,0.63
2022-12-31 18:30:00,8.0,3.83,0.57
2022-12-31 19:30:00,6.0,3.88,0.82


In [43]:
# converte as colunas para float
df = df.astype(float)
df

,PM2.5,PM10,Monóxido de Carbono
Data e Hora,,,
2015-11-04 10:30:00,25.0,39.41,0.43
2015-11-04 11:30:00,13.0,38.38,0.42
2015-11-04 12:30:00,12.0,24.91,0.57
2015-11-04 13:30:00,14.0,32.16,0.55
2015-11-04 14:30:00,12.0,26.99,0.56
...,...,...,...
2022-12-31 17:30:00,3.0,5.25,0.63
2022-12-31 18:30:00,8.0,3.83,0.57
2022-12-31 19:30:00,6.0,3.88,0.82


In [44]:
# normalizando os dados manualmente

def manual_normalization(data):
    normalizado = data.copy()
    min_max_values = {}
    for column in df.columns:
        min_val = df[column].min()
        max_val = df[column].max()
        min_max_values[column] = (min_val, max_val)

        # Normaliza os valores da coluna
        normalizado[column] = (df[column] - min_val) / (max_val - min_val)
    
    return normalizado, min_max_values

df, max_min_values = manual_normalization(df)

df


,PM2.5,PM10,Monóxido de Carbono
Data e Hora,,,
2015-11-04 10:30:00,0.167832,0.048663,0.041992
2015-11-04 11:30:00,0.083916,0.047375,0.041016
2015-11-04 12:30:00,0.076923,0.030524,0.055664
2015-11-04 13:30:00,0.090909,0.039594,0.053711
2015-11-04 14:30:00,0.076923,0.033126,0.054688
...,...,...,...
2022-12-31 17:30:00,0.013986,0.005930,0.061523
2022-12-31 18:30:00,0.048951,0.004153,0.055664
2022-12-31 19:30:00,0.034965,0.004216,0.080078


In [45]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs, dtype=np.float32), np.array(ys, dtype=np.float32)


seq_length = 8
X, y = create_sequences(df.values, seq_length)


In [46]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(33622, 8, 3)
(7205, 8, 3)
(7205, 8, 3)


In [47]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float().reshape(-1, 1)
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float().reshape(-1, 1)
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float().reshape(-1, 1)

train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [48]:
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

input_size = X_train.shape[2]  # Supondo que X_train seja um tensor com a forma (n_samples, seq_len, n_features)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Definição do modelo LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, drop_prob_input, drop_prob_recurrent,
                 activation_function):
        super(LSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.dropout_input = nn.Dropout(drop_prob_input)
        self.dropout_recurrent = nn.Dropout(drop_prob_recurrent)
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.activation_function = activation_function

    def forward(self, input_seq):
        input_seq = self.dropout_input(input_seq)
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1, :])
        if self.activation_function == 'relu':
            predictions = nn.ReLU()(predictions)
        elif self.activation_function == 'sigmoid':
            predictions = nn.Sigmoid()(predictions)
        elif self.activation_function == 'tanh':
            predictions = nn.Tanh()(predictions)
        return predictions


def train_and_evaluate_lstm(input_size, hidden_layer_size, num_layers, lr, batch_size, drop_prob_input,
                            drop_prob_recurrent, activation_function,
                            weight_decay, num_epochs, patience):
    model = LSTM(
        input_size=input_size,
        hidden_layer_size=hidden_layer_size,
        num_layers=num_layers,
        drop_prob_input=drop_prob_input,
        drop_prob_recurrent=drop_prob_recurrent,
        output_size=1,
        activation_function=activation_function
    ).to(device)
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_data = TensorDataset(X_train, y_train)
    val_data = TensorDataset(X_val, y_val)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

    # Early stopping 
    best_val_loss = np.inf
    epochs_no_improve = 0
    val_losses = []

    def train_model():
        model.train()
        for seq, labels in train_loader:
            seq, labels = seq.to(device), labels.to(device)
            optimizer.zero_grad()
            y_pred = model(seq)
            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

    def evaluate_model():
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for seq, labels in val_loader:
                seq, labels = seq.to(device), labels.to(device)
                y_pred = model(seq)
                single_loss = loss_function(y_pred, labels)
                val_loss += single_loss.item() * seq.size(0)
        val_loss /= len(val_loader.dataset)
        return val_loss

    for epoch in range(num_epochs):
        train_model()
        val_loss = evaluate_model()
        val_losses.append(val_loss)
        # print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "./best_model_lstm.pth")
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print("Early stopping after {} epochs".format(epoch + 1))
                break

    return model, val_losses


from sklearn.base import BaseEstimator, RegressorMixin


class PyTorchLSTMRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_layer_size=2, num_layers=2, lr=0.001, batch_size=64, drop_prob_input=1.0,
                 drop_prob_recurrent=1.0,
                 activation_function='relu', weight_decay=1e-8, num_epochs=5, patience=20):
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lr = lr
        self.drop_prob_input = drop_prob_input
        self.drop_prob_recurrent = drop_prob_recurrent
        self.batch_size = batch_size
        self.activation_function = activation_function
        self.weight_decay = weight_decay
        self.num_epochs = num_epochs
        self.patience = patience

    def fit(self, X, y):
        self.model, self.val_losses = train_and_evaluate_lstm(
            input_size=input_size,
            hidden_layer_size=self.hidden_layer_size,
            num_layers=self.num_layers,
            lr=self.lr,
            batch_size=self.batch_size,
            drop_prob_input=self.drop_prob_input,
            drop_prob_recurrent=self.drop_prob_recurrent,
            activation_function=self.activation_function,
            weight_decay=self.weight_decay,
            num_epochs=self.num_epochs,
            patience=self.patience
        )
        return self

    def predict(self, X):
        self.model.eval()
        test_data = torch.from_numpy(X).float()
        test_loader = DataLoader(test_data, batch_size=self.batch_size, shuffle=False)

        predictions = []
        with torch.no_grad():
            for seq in test_loader:
                seq = seq.to(device)
                y_pred = self.model(seq)
                predictions.append(y_pred.cpu().numpy())

        predictions = np.concatenate(predictions, axis=0)
        return predictions


# Parâmetros para busca
param_space = {
    'hidden_layer_size': Integer(8, 512),
    'lr': Real(0.0001, 0.1, prior='log-uniform'),
    'num_layers': Integer(1, 10),
    'batch_size': Categorical([64, 96, 128]),
    'num_epochs': Categorical([400]),
    'activation_function': Categorical(['relu', 'sigmoid', 'tanh']),
    'drop_prob_input': Real(0.0, 0.5),
    'drop_prob_recurrent': Real(0.0, 0.5),
    'weight_decay': Real(1e-8, 1e-3, prior='log-uniform'),
}

tscv = TimeSeriesSplit(n_splits=3)

# Usando BayesSearchCV
bayes_search = BayesSearchCV(estimator=PyTorchLSTMRegressor(), search_spaces=param_space,
                             scoring='neg_mean_squared_error', cv=tscv, n_iter=100, random_state=42, verbose=3)
bayes_search.fit(X_train.numpy(), y_train.numpy())


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Early stopping after 26 epochs
[CV 1/3] END activation_function=sigmoid, batch_size=128, drop_prob_input=0.46643399942391695, drop_prob_recurrent=0.15789979674352436, hidden_layer_size=346, lr=0.0017472534129202037, num_epochs=400, num_layers=8, weight_decay=3.3290211565094177e-07;, score=-0.003 total time= 1.4min
Early stopping after 50 epochs
[CV 2/3] END activation_function=sigmoid, batch_size=128, drop_prob_input=0.46643399942391695, drop_prob_recurrent=0.15789979674352436, hidden_layer_size=346, lr=0.0017472534129202037, num_epochs=400, num_layers=8, weight_decay=3.3290211565094177e-07;, score=-0.003 total time= 2.7min
Early stopping after 35 epochs
[CV 3/3] END activation_function=sigmoid, batch_size=128, drop_prob_input=0.46643399942391695, drop_prob_recurrent=0.15789979674352436, hidden_layer_size=346, lr=0.0017472534129202037, num_epochs=400, num_layers=8, weight_decay=3.3290211565094177e-07;, score=-0.003 total time= 

KeyboardInterrupt: 

In [ ]:
print("Best Parameters:", bayes_search.best_params_)
print("Best Score:", -bayes_search.best_score_)
# MAPE, RMSE, MAE
print("MAPE:", np.mean(
    np.abs(bayes_search.best_estimator_.predict(X_val.numpy()) - y_val.numpy()) / np.abs(y_val.numpy()) * 100))
print("RMSE:", np.sqrt(np.mean((bayes_search.best_estimator_.predict(X_val.numpy()) - y_val.numpy()) ** 2)))
print("MAE:", np.mean(np.abs(bayes_search.best_estimator_.predict(X_val.numpy()) - y_val.numpy())))
print("R2:",
      np.corrcoef(bayes_search.best_estimator_.predict(X_val.numpy()).flatten(), y_val.numpy().flatten())[0, 1] ** 2)

In [ ]:
plt.plot(bayes_search.best_estimator_.val_losses)
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Validation Loss over Epochs')
plt.show()

In [ ]:
# Treinar o modelo com os melhores parâmetros encontrados
best_params = bayes_search.best_params_
print("Training with best parameters...")
print(best_params)

In [ ]:


model, val_losses = train_and_evaluate_lstm(input_size=input_size, **best_params, patience=5)

# Fazer predições
model.eval()
with torch.no_grad():
    y_pred = model(X_val.to(device)).cpu().numpy()

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(y_val.numpy(), label='Valores Reais', color='black', alpha=1)
plt.plot(y_pred, label='Valores de Previsão', color='green', alpha=0.5)
plt.xlabel('Amostras')
plt.ylabel('Valores')
plt.title('Valores Reais vs. Valores de Previsão')
plt.legend()
plt.show()